In [ ]:
import os
import random
import numpy as np
import nengo
import skspeech
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
plt.rc('figure', figsize=(14, 4))

from mpl_toolkits.mplot3d import Axes3D
from nengo.dists import Choice, Uniform

In [ ]:
# Syllable basis functions:
# onset, nucleus, coda (could just be Gaussian tiled, but whatever)

def cart2pol(x1, x2):
    return np.arctan2(x2, x1) / (2 * np.pi) + 0.5

def pol2cart(angle, amplitude=1):
    return (amplitude * np.cos(angle * 2 * np.pi),
            amplitude * np.sin(angle * 2 * np.pi))

def onset_f(t):
    return 1. if 0.05 < t < 0.4 else 0.

def nucleus_f(t):
    return 1. if 0.3 < t < 0.8 else 0.

def coda_f(t):
    return 1. if 0.7 < t < 0.9 else 0.

def run(func, angle):
    out = np.zeros_like(angle)
    for i, theta in enumerate(angle):
        out[i] = func(theta)
    return out

t = np.linspace(0, 1)
plt.plot(t, run(onset_f, t))
plt.plot(t, run(nucleus_f, t))
plt.plot(t, run(coda_f, t))

In [ ]:
def cart2pol(fn):
    def _fn(x):
        return fn(np.arctan2(x[1], x[0]) / (2 * np.pi) + 0.5)
    _fn.__name__ = fn.__name__
    return _fn

In [ ]:
# Get a gesture score
dt = 0.001
gs = skspeech.vtl.parse_ges('ges-de-cvc/das.ges')
print gs.t_end
traj = gs.trajectory(dt=dt)
plt.pcolormesh(traj.T)

In [ ]:
# Do some direct mode simulations, convince myself it works

tau = 0.1   # Post-synaptic time constant for feedback
w_max = 25.1327  # Maximum frequency is w_max/(2*pi) = 4 Hz

with nengo.Network() as net:
    # This mostly works when using Direct mode, but not otherwise...
    net.config[nengo.Ensemble].neuron_type = nengo.Direct()

    # --- Controlled oscillator
    osc = nengo.Ensemble(500, dimensions=3, radius=1.7)
    
    # The feedback connection
    def feedback(x):
        x0, x1, w = x  # These are the three variables stored in the ensemble
        return x0 - w*w_max*tau*x1, x1 + w*w_max*tau*x0, 0
    nengo.Connection(osc, osc, function=feedback, synapse=tau)
    
    freq = nengo.Ensemble(30, dimensions=1)
    nengo.Connection(freq, osc[2], synapse=None)
    kick = nengo.Node(lambda t: -1. / 0.18 if t < 0.015 else 0)
    nengo.Connection(kick, osc[0])
    # Scale freq to gesture trajectory (but a bit slower)
    input_freq = nengo.Node((1. / gs.t_end) * 0.245)
    nengo.Connection(input_freq, freq)

    # --- Get syllable basis
    syllparts = nengo.networks.EnsembleArray(50, n_ensembles=3)
    onset = syllparts.output[0]
    nucleus = syllparts.output[1]
    coda = syllparts.output[2]
    nengo.Connection(osc, syllparts.input[0], function=cart2pol(onset_f))
    nengo.Connection(osc, syllparts.input[1], function=cart2pol(nucleus_f))
    nengo.Connection(osc, syllparts.input[2], function=cart2pol(coda_f))

    # --- Represent gesture trajectory
    g_dims = traj.shape[1]
    g_traj = nengo.Ensemble(30*g_dims, dimensions=g_dims)
    
    def traj_f(t):
        ix = int(t / dt) % traj.shape[0]
        return traj[ix]
    nengo.Connection(nengo.Node(traj_f), g_traj)

    # --- Associate gesture with part of syllable with product
    sp_in_ed = syllparts.n_ensembles + 1
    sp_in = nengo.networks.EnsembleArray(30, n_ensembles=g_dims, ens_dimensions=sp_in_ed,
                                         intercepts=Uniform(0.1, 0.4))
    nengo.Connection(g_traj, sp_in.input[::sp_in_ed])  # traj -> every sp_in_ed dims
    # Connect syllable bases every sp_in_ed dims
    for d in xrange(g_dims):
        nengo.Connection(syllparts.output,
                         sp_in.input[d*sp_in_ed+1:(d+1)*sp_in_ed])

    # Output products beteen dim 0 and all rest
    def sp_in_prod(x):
        return x[0] * x[1:]
    sp_out = sp_in.add_output('sp_out', sp_in_prod)

    # --- Integrate each syllable part.
    #     The SP is where it ends up.
    sp = nengo.networks.EnsembleArray(30, n_ensembles=g_dims * syllparts.n_ensembles,
                                      intercepts=Uniform(0.1, 0.4), encoders=Choice([[1]]))
    nengo.Connection(sp.output, sp.input,
                     transform=np.identity(sp.dimensions), synapse=tau)
    nengo.Connection(sp_out, sp.input, transform=1. / tau, synapse=tau)

    p_osc = nengo.Probe(osc, synapse=0.01)
    p_syllparts = nengo.Probe(syllparts.output, synapse=0.01)
    p_traj = nengo.Probe(g_traj, synapse=0.01)
    p_sp_in = nengo.Probe(sp_in.output, synapse=0.01)
    p_sp = nengo.Probe(sp.output, synapse=0.01)

In [ ]:
sim = nengo.Simulator(net)
sim.run(gs.t_end)

g = 0 # Zoom in on a gesture
print(gs.labels[g])

plt.plot(sim.data[p_osc][:, 0], sim.data[p_osc][:, 1])
plt.figure()
plt.plot(sim.trange(), sim.data[p_syllparts])
plt.figure()
plt.pcolormesh(sim.data[p_traj].T)
plt.colorbar()
plt.figure()
# plt.pcolormesh(sim.data[p_sp_in].T[g*4:(g+1)*4])
plt.pcolormesh(sim.data[p_sp_in].T)
plt.colorbar()
plt.figure()
# plt.pcolormesh(sim.data[p_sp].T[g*3:(g+1)*3])
plt.pcolormesh(sim.data[p_sp].T)
plt.colorbar()

In [ ]:
print(sum(e.n_neurons for e in net.all_ensembles))

In [ ]:
# Basic premise works! Let's generate a set of SPs for some syllables
dt = 0.001
gscores = {}

# Split the utterance into 

for ges in os.listdir('ges-de-cvc'):
    gscores[ges[:-4]] = skspeech.vtl.parse_ges('ges-de-cvc/%s' % ges)

print list(gscores)

def traj2pointer(traj):
    """Convert a gesture trajectory to a semantic pointer.

    We do this by determining whether each gesture is done
    in the onset, nucleus, or coda of the syllable.
    In the end we get a binary vector of whether each gesture
    occurred in that part of the syllable.
    """
    t_end = traj.shape[0]
    out = np.zeros(traj.shape[1] * 3)
    onoff = np.diff(np.vstack(
        [np.zeros(traj.shape[1]), traj, np.zeros(traj.shape[1])]), axis=0)
    onoff[onoff < 0] = -1  # binarize
    onoff[onoff > 0] = 1  # binarize
    for i in xrange(traj.shape[1]):
        midpoints = np.where(onoff[:, i] == -1)[0] - np.where(onoff[:, i] == 1)[0]
        for midpoint in midpoints:
            # Snap it to which third it resides in
            assert midpoint > 0
            ix = int(3 * float(midpoint) / t_end)
            assert 0 <= ix <= 2
            out[(i*3)+ix] = 1
    return out

pointers = {}

# For testing, just use the first three
for ges, gs in gscores.items()[:3]:
    pointers[ges] = traj2pointer(gs.trajectory(dt=dt))

# Make sure 'das' is in there too
pointers['das'] = traj2pointer(gscores['das'].trajectory(dt=dt))

In [ ]:
# Make sure no pointers are identical
for g1, p1 in pointers.items():
    for g2, p2 in pointers.items():
        assert g1 == g2 or not np.allclose(p1, p2)

In [ ]:
# Plot a random pointer
ges = random.choice(list(pointers))
plt.title("SP for '%s'" % ges)
plt.stem(pointers[ges])

In [ ]:
# Use these SPs to build an associative memory
sps = [pointers[key][np.newaxis, :] for key in sorted(list(pointers))]
sps = np.concatenate(sps, axis=0)
print sps.shape
# Outputs are initially set to zero.
# Later, we will impose the trajectory
# by making each output ensemble an oscillator
# with the trajectory decoded as it oscillates.
out = np.zeros((sps.shape[0], sps.shape[1] / 3))
print out.shape

with net:
    net.config[nengo.Ensemble].neuron_type = nengo.LIFRate()
    syllabary = nengo.networks.AssociativeMemory(
        input_vectors=sps, output_vectors=out, threshold_output=True)
    nengo.Connection(sp.output, syllabary.input)
    
    # Probe the activity of the ensembles; we'll look at the mean over time
    p_syll = {}
    for i, ges in enumerate(sorted(pointers)):
        p_syll[ges] = nengo.Probe(
            syllabary.thresh_ens.ea_ensembles[i].neurons)

In [ ]:
sim = nengo.Simulator(net)
sim.run(gs.t_end)

plt.figure()
plt.stem(np.clip(sim.data[p_sp][-1], 0, 1))
plt.figure()
plt.stem(pointers['das'])

for ges, probe in p_syll.items():
    print("Mean of '%s': %f" % (ges, np.mean(sim.data[probe])))